In [2]:
# import packages 
import pandas as pd                   # dataframe
import matplotlib.pyplot as plt       # graphs & other analysis
import matplotlib as mpl              # graphics parameters
import numpy as np                    # numerical calculations 
import datetime as dt                 # data/ time
%matplotlib inline
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

import urllib.request
import json
import bs4

In [3]:
# Burgerking Korea Parsing the data from the website

BK_url = 'http://www.burgerking.co.kr/api/store/searchmap/empty/?areacd='
response = urllib.request.urlopen(BK_url)
data_BK = json.loads(response.read().decode('utf-8'))
data_BK

[{'AllHour': 'N',
  'CloseCleaning': '',
  'ClosePeakSeason': '',
  'CloseWeekday': '22:00',
  'Delivery': 'N',
  'DriveThrough': 'N',
  'Morning': 'N',
  'NewAddr': '서울특별시 금천구 가산디지털1로 ',
  'NewAddr2': '168 우림라이온스밸리 A동',
  'OpenTime': '10:00',
  'PhoneNumber': '02-853-0332',
  'PointX': 37.4799652,
  'PointY': 126.882637,
  'StoreNM': '가산디지털점',
  'StoreSQ': 1},
 {'AllHour': 'N',
  'CloseCleaning': '',
  'ClosePeakSeason': '/ 토: 10시 ~ 23시 / 일: 10시 ~ 22시',
  'CloseWeekday': ' 22시 /  금: 11시~23시',
  'Delivery': 'N',
  'DriveThrough': 'N',
  'Morning': 'N',
  'NewAddr': '서울특별시 금천구 디지털로10길',
  'NewAddr2': '9 HYHILL 6층',
  'OpenTime': '월~목: 11시',
  'PhoneNumber': '02-2136-9962',
  'PointX': 37.47762,
  'PointY': 126.889053,
  'StoreNM': '가산현대시티아울렛점',
  'StoreSQ': 2},
 {'AllHour': 'N',
  'CloseCleaning': '',
  'ClosePeakSeason': '',
  'CloseWeekday': '23:30',
  'Delivery': 'N',
  'DriveThrough': 'N',
  'Morning': 'N',
  'NewAddr': '서울특별시 강서구 양천로',
  'NewAddr2': '559 이마트3층 (가양동)',
  'OpenTime':

In [4]:
df= pd.DataFrame(data_BK)
df.head(3)

,AllHour,CloseCleaning,ClosePeakSeason,CloseWeekday,Delivery,DriveThrough,Morning,NewAddr,NewAddr2,OpenTime,PhoneNumber,PointX,PointY,StoreNM,StoreSQ
0,N,,,22:00,N,N,N,서울특별시 금천구 가산디지털1로,168 우림라이온스밸리 A동,10:00,02-853-0332,37.479965,126.882637,가산디지털점,1
1,N,,/ 토: 10시 ~ 23시 / 일: 10시 ~ 22시,22시 / 금: 11시~23시,N,N,N,서울특별시 금천구 디지털로10길,9 HYHILL 6층,월~목: 11시,02-2136-9962,37.477620,126.889053,가산현대시티아울렛점,2
2,N,,,23:30,N,N,N,서울특별시 강서구 양천로,559 이마트3층 (가양동),10:00,02-3664-0221,37.558193,126.861816,가양이마트점,3


In [5]:
temp_table = df[['NewAddr','StoreNM']]

In [6]:
temp_table.head()

,NewAddr,StoreNM
0,서울특별시 금천구 가산디지털1로,가산디지털점
1,서울특별시 금천구 디지털로10길,가산현대시티아울렛점
2,서울특별시 강서구 양천로,가양이마트점
3,서울특별시 서초구 잠원로,강남NC점
4,서울특별시 서초구 사평대로,강남교보점


In [7]:
foo = lambda x: pd.Series([i for i in x.split(' ')[:2]])
rev = temp_table['NewAddr'].apply(foo)
rev.rename(columns={0:'City',1:'Province'},inplace=True)
BK_table = pd.concat([temp_table, rev], axis=1)
del BK_table['NewAddr']
BK_table.head(3)
# Alternative
# burgerking_table2= pd.DataFrame(bgk_tbl['NewAddr'].apply(lambda v: v.split()[:2]).tolist(),columns=('si', 'gu'))

,StoreNM,City,Province
0,가산디지털점,서울특별시,금천구
1,가산현대시티아울렛점,서울특별시,금천구
2,가양이마트점,서울특별시,강서구


In [8]:
BK_table['City'].unique()

array(['서울특별시', '서울시', '경기도', '수원시', '안산시', '인천광역시', '인천시', '인천', '강릉시',
       '강원도', '충청남도', '충남', '대전광역시', '대전시', '충청북도', '청주시', '세종특별자치시',
       '부산광역시', '울산광역시', '울산시', '대구광역시', '대구시', '경북', '경상북도', '포항시',
       '경상남도', '전라남도', '전남', '광주광역시', '전라북도', '전북', '제주특별자치도'], dtype=object)

In [9]:
city_renaming = """서울시:서울특별시 충남:충청남도 강원:강원도 경기:경기도 수원시:경기도 충북:충청북도 경남:경상남도 경북:경상북도
전남:전라남도 전북:전라북도 제주도:제주특별자치도 제주:제주특별자치도 대전시:대전광역시 대구시:대구광역시 인천시:인천광역시 인천:인천광역시
광주시:광주광역시 울산시:울산광역시"""
city_aliases = dict((aliasset.split(':') for aliasset in city_renaming.split()))
city_aliases


{'강원': '강원도',
 '경기': '경기도',
 '경남': '경상남도',
 '경북': '경상북도',
 '광주시': '광주광역시',
 '대구시': '대구광역시',
 '대전시': '대전광역시',
 '서울시': '서울특별시',
 '수원시': '경기도',
 '울산시': '울산광역시',
 '인천': '인천광역시',
 '인천시': '인천광역시',
 '전남': '전라남도',
 '전북': '전라북도',
 '제주': '제주특별자치도',
 '제주도': '제주특별자치도',
 '충남': '충청남도',
 '충북': '충청북도'}

In [10]:
BK_table['City'] = BK_table['City'].apply(lambda x: city_aliases.get(x, x))

In [11]:
BK_table['City'].unique()

array(['서울특별시', '경기도', '안산시', '인천광역시', '강릉시', '강원도', '충청남도', '대전광역시',
       '충청북도', '청주시', '세종특별자치시', '부산광역시', '울산광역시', '대구광역시', '경상북도', '포항시',
       '경상남도', '전라남도', '광주광역시', '전라북도', '제주특별자치도'], dtype=object)

In [13]:
BK_table['Province'].unique()

array(['금천구', '강서구', '서초구', '강남구', '광진구', '성북구', '관악구', '구로구', '용산구',
       '강동구', '동작구', '영등포구', '중구', '양천구', '송파구', '은평구', '안산시', '마포구',
       '서대문구', '도봉구', '종로구', '중랑구', '동대문구', '고양시', '과천시', '수원시', '광명시',
       '광주시', '구리시', '파주시', '김포시', '동두천시', '화성시', '부천시', '성남시', '군포시',
       '영통구', '용인시', '단원구', '여주군', '오산시', '의정부시', '이천시', '안양시', '평택시',
       '하남시', '남구', '부평구', '연수구', '송도국제대로', '남동구', '계양구', '하슬라로', '홍천군',
       '속초시', '원주시', '당진시', '아산시', '천안시', '유성구', '서구', '동구', '청주시', '흥덕구',
       '청원군', '충주시', nan, '부산진구', '해운대구', '수성구', '달서구', '상주시', '경산시',
       '경주시', '구미시', '안동시', '포항시', '북구', '거제시', '창원시', '김해시', '진주시', '나주시',
       '목포시', '순천시', '여수시', '무안군', '광산구', '군산시', '익산시', '전주시', '제주시'], dtype=object)

In [16]:
Burgerking = BK_table.apply(lambda r: r['City'] + ' ' + str(r['Province']), axis=1)

In [17]:
B = Burgerking.value_counts()

In [19]:
B.head()

서울특별시 강남구    20
경기도 수원시      11
서울특별시 서초구     9
서울특별시 중구      7
경기도 성남시       7
dtype: int64

In [20]:
kfc_dists = "강원 경기 경남 경북 광주 대구 대전 부산 서울 울산 인천 전남 전북 제주 충남 충북".split()

In [21]:
KFC_DISTSEARCH_URL = 'http://www.kfckorea.com/store/store_addr_search.asp?addr_div=gugun&sido={location}'

def kfc_search_subdists(location):
    response = urllib.request.urlopen(
        KFC_DISTSEARCH_URL.format(location=urllib.parse.quote(location.encode('utf-8'))))
    kfc_data = response.read().decode('utf-8')
    soup = bs4.BeautifulSoup(kfc_data)
    return list(filter(None, [tag.attrs['value'] for tag in soup.findAll('option')]))

In [22]:
kfc_alldist = [(d, subd) for d in kfc_dists for subd in kfc_search_subdists(d)]
kfc_alldist[:5], len(kfc_alldist)

c:\users\dongjin\envs\03stat\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


([('강원', '강릉시'),
  ('강원', '원주시'),
  ('강원', '정선군'),
  ('강원', '춘천시'),
  ('경기', '고양시 덕양구')],
 106)

In [23]:
KFC_STORESEARCH_URL = ('http://www.kfckorea.com/store/store_search.asp?sales_24_yn_=&'
                       'sales_wifi_yn_=&sales_order_group_yn_=&sales_park_yn_=&sales_subway_yn_=&'
                       'sales_mart_in_yn_=&searchFlag=0&addr_div1={div1}&addr_div2={div2}&keyword=')

def kfc_search_stores_in_dist(d1, d2):
    response = urllib.request.urlopen(
        KFC_STORESEARCH_URL.format(div1=urllib.parse.quote(d1.encode('utf-8')),
                                   div2=urllib.parse.quote(d2.encode('utf-8'))))
    return json.loads(response.read().decode('utf-8'))['store']

In [24]:
found = []
for d1, d2 in kfc_alldist:
    found.extend(kfc_search_stores_in_dist(d1, d2))
kfc_tbl = pd.DataFrame(found)

In [25]:
kfc_tbl.head()

,addr_div1,addr_div2,clean_day,map_url1,map_url2,new_addr1,old_addr1,phone,post_no,sales_24_yn,...,sales_park_yn,sales_subway_yn,sales_time,sales_wifi_yn,shop_code,shop_location,shop_name,shop_seq,shop_use_div,use_area
0,강원,강원,,37.76419798981757,128.87763186572403,강원도 강릉시 솔올로 25 현진빌딩 1층 (교동),강원도 강릉시 교동 1882-3 현진빌딩 1층,033)642-6332,210-100,N,...,N,N,10:00 ~ 22:00,N,1410723,교동주공 3단지 308동 옆,강릉교동,162,Y,A
1,강원,강원,,37.3206210,127.9190290,강원도 원주시 북원로 1928 (무실동) 원주이마트 지하1층 푸드코트 內,강원도 원주시 무실동 1062-1 원주이마트,033)761-6420,26395,N,...,N,N,10:00 ~ 22:00,N,1410817,원주이마트 지하1층 푸드코트 內,원주이마트,259,Y,A
2,강원,강원,10:00~22:00,37.2039550,128.8370300,강원도 고한읍 고한리 산289-6 하이원리조트 밸리스키 하우스,강원 정선군 고한읍 고한리 산289-6 하이원리조트 밸리스키 하우스 3층,033)591-9749,233-901,N,...,Y,N,유동적(시즌에 따라),Y,1410696,하이원리조트 벨리스키하우스 3층,하이원리조트,29,Y,A
3,강원,강원,,37.8684840,127.7456640,강원도 춘천시 영서로 2027 (석사동),강원 춘천시 석사동 885-1번지,033)912-3311,24394,N,...,N,N,10:00 ~ 22:00,N,1410800,,춘천석사,241,Y,A
4,강원,강원,,37.863814444176135,127.71834104991627,강원도 춘천시 경춘로 2341 이마트 춘천점 1층 푸드코트 內,강원도 춘천시 온의동 511번지 이마트 춘천점 1층 푸드코트 內,033)252-3369,200-938,N,...,Y,N,10:00~22:00,N,1410714,이마트 춘천점 1층 푸드코트 내,춘천이마트,152,Y,A


In [26]:
kfc_locs = pd.DataFrame(kfc_tbl['old_addr1'].apply(
    lambda v: v.replace('&nbsp;', ' ').replace('&#160;', ' ').replace('광주 광역', '광주광역').split()[:2]).tolist(),
              columns=('City', 'Province'))
kfc_locs['d1'].unique()

KeyError: 'd1'

In [ ]:
d1_aliases = """서울시:서울특별시 충남:충청남도 강원:강원도 경기:경기도 충북:충청북도 경남:경상남도 경북:경상북도
전남:전라남도 전북:전라북도 제주도:제주특별자치도 제주:제주특별자치도 대전시:대전광역시 대구시:대구광역시 인천시:인천광역시
광주시:광주광역시 울산시:울산광역시 광주:광주광역시 대구:대구광역시 대전:대전광역시 부산:부산광역시 부산시:부산광역시
인천:인천광역시 서울:서울특별시 울산:울산광역시"""
d1_aliases = dict(aliasset.split(':') for aliasset in d1_aliases.split())
kfc_locs['City'] = kfc_locs['City'].apply(lambda v: d1_aliases.get(v, v))
kfc_locs['City'].unique()

In [ ]:
kfc_locs['Province'].unique()

In [ ]:
K = kfc_locs.apply(lambda r: r['City'] + ' ' + r['Province'], axis=1).value_counts()
K.head()

In [ ]:
BMK = pd.DataFrame({'B': B, 'K': K}).fillna(0)
BMK['total'] = BMK.sum(axis=1)
BMK = BMK.sort('total', ascending=False)
BMK.head(10)

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import rcParams, style
style.use('ggplot')

In [ ]:
plt.figure(figsize=(4, 3))
BMK.sum(axis=0).iloc[:2].plot(kind='bar')

In [ ]:
import scipy.stats

In [ ]:
fig = plt.figure(figsize=(6, 3))

def plot_nstores(b1, b2, label1, label2):
    plt.scatter(BMK[b1] + np.random.random(len(BMK)),
                BMK[b2] + np.random.random(len(BMK)),
                )
    plt.xlim(-1, 15)
    plt.ylim(-1, 15)
    plt.xlabel(label1)
    plt.ylabel(label2)
    
    r = scipy.stats.pearsonr(BMK[b1], BMK[b2])
    plt.annotate('r={:.3f}'.format(r[0]), (10, 12.5))


ax = fig.add_subplot()
plot_nstores('B', 'K', 'Burger King', 'KFC')


plt.tight_layout()

In [ ]:
plt.figure(figsize=(4, 3))
for col, label in [('B', 'Burger King'), ('K', 'KFC')]:
    cumulv = np.cumsum(sorted(BMK[col], reverse=True)) / BMK[col].sum()
    plt.plot(cumulv, label='{} ({})'.format(label, int(BMK[col].sum())))
plt.legend(loc='best')
plt.xlabel('Number of districts (si/gun/gu)')
plt.ylabel('Cumulative fraction')

In [ ]:
LOTTERIA_URL = 'http://www.lotteria.com/Shop/Shop_Ajax.asp'
LOTTERIA_VALUES = {
    'Page': 1, 'PageSize': 2000, 'BlockSize': 2000,
    'SearchArea1': '', 'SearchArea2': '', 'SearchType': "TEXT",
    'SearchText': '', 'SearchIs24H': '', 'SearchIsWifi': '',
    'SearchIsDT': '', 'SearchIsHomeService': '', 'SearchIsGroupOrder': '',
    'SearchIsEvent': ''}
LOTTERIA_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.10; rv:12.0) Gecko/20100101',
    'Host': 'www.lotteria.com',
    'Accept': 'text/html, */*; q=0.01',
    'Accept-Language': 'en-us,en;q=0.5',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Referer': 'http://www.lotteria.com/Shop/Shop_List.asp?Page=1&PageSize=2000&BlockSize=2000&Se'
               'archArea1=&SearchArea2=&SearchType=TEXT&SearchText=&SearchIs24H=&SearchIsWifi=&Se'
               'archIsDT=&SearchIsHomeService=&SearchIsGroupOrder=&SearchIsEvent=',
}

postdata = urllib.parse.urlencode(LOTTERIA_VALUES).encode('utf-8')
req = urllib.request.Request(LOTTERIA_URL, postdata, LOTTERIA_HEADERS)
response = urllib.request.urlopen(req)
ltr_data = response.read().decode('utf-8')
soup = bs4.BeautifulSoup(ltr_data)

In [ ]:
found = []
for tag in soup.findAll('tr', {'class': 'shopSearch'}):
    subtag = [tag.findAll('td', {'style': 'padding-right:10px;'}
                          )[i].contents[0].contents[0]
              for i in (0, 1)]
    found.append([subtag[0]] + subtag[1].replace('광주 광역', '광주광역').split()[:2])
ltr_tbl = pd.DataFrame(found, columns=('storename', 'd1', 'd2'))
ltr_tbl.head()

In [ ]:
ltr_tbl['d1'].unique()

In [ ]:
d1_aliases = """강원:강원도 충북:충청북도 부산:부산광역시 경기:경기도 전남:전라남도 경북:경상북도
대구:대구광역시 서울:서울특별시 세종:세종특별자치시 경남:경상남도 충남:충청남도 대전:대전광역시
울산:울산광역시 제주:제주특별자치도 인천:인천광역시 전북:전라북도 광주:광주광역시 대전시:대전광역시
충남시:충청남도 청북:충청북도"""
d1_aliases = dict(aliasset.split(':') for aliasset in d1_aliases.split())
ltr_tbl['d1'] = ltr_tbl['d1'].apply(lambda v: d1_aliases.get(v, v))
ltr_tbl['d1'].unique()

In [ ]:
ltr_tbl['d2'].unique()

In [ ]:
d2_aliases = """나주시금천면:나주시 장성군장성읍:장성군 고흥군고흥읍:고흥군 기장군정관면:기장군
창녕군남지읍:창녕군 임실군임실읍:임실군 원주시지정면:원주시 진구:부산진구 논산시연무읍:논산시"""
d2_aliases = dict(aliasset.split(':') for aliasset in d2_aliases.split())
ltr_tbl['d2'] = ltr_tbl['d2'].apply(lambda v: d2_aliases.get(v, v))
ltr_tbl['d2'].unique()

In [ ]:
ltr_tbl[ltr_tbl['d2'].apply(lambda v: v[-1] not in '시군구')]

In [ ]:
d2_aliases = """연기면:세종시 금남면:세종시 조치원읍:세종시 아산신:아산시 구로:구로구
종로2가:종로구"""
d2_aliases = dict(aliasset.split(':') for aliasset in d2_aliases.split())
ltr_tbl['d2'] = ltr_tbl['d2'].apply(lambda v: d2_aliases.get(v, v))
ltr_tbl['d2'].unique()

In [ ]:
L = ltr_tbl.apply(lambda r: r['d1'] + ' ' + r['d2'], axis=1).value_counts()
L.head()

In [ ]:
distr_latlon = pd.read_table('lonlat.csv')
distr_latlon.head()

In [ ]:
distr_latlon.index = distr_latlon.apply(lambda r: r['d1'] + ' ' + r['d2'], axis=1)

In [ ]:
bgt = pd.DataFrame({'B': B, 'K': K, 'L': L}).fillna(0)
bgt = pd.merge(distr_latlon, bgt, how='outer', left_index=True, right_index=True)
bgt.head()

In [ ]:
# 행정구역 정보가 없는데 버거 매장은 있는 지역을 찾아 봄.
bgt[np.isnan(bgt['area'])]

In [ ]:
bgidx_cols = ['B', 'K', 'L']
bgt.loc['경기도 여주시', bgidx_cols] += bgt.loc['경기도 여주군', bgidx_cols]
bgt.loc['울산광역시 울주군', bgidx_cols] += bgt.loc['울산광역시 을주군', bgidx_cols]
bgt.loc['충청남도 천안시', bgidx_cols] += bgt.loc['충청북도 천안시', bgidx_cols]
bgt.loc['충청북도 청주시', bgidx_cols] += bgt.loc['충청북도 청원군', bgidx_cols] # 2014년 7월 1일 통합.
bgt = bgt[~np.isnan(bgt['area'])].fillna(0)

In [ ]:
bgt.head()

In [ ]:
def short_distr(name):
    wide, narrow = name.split()
    if narrow.endswith('구'):
        return wide[:2] + (narrow[:-1] if len(narrow) > 2 else narrow)
    elif narrow == '고성군': # 고성군은 강원도, 경상남도에 있다.
        return '고성({})'.format({'강원도': '강원', '경상남도': '경남'}[wide])
    else:
        return narrow[:-1]

bgt['shortname'] = list(map(short_distr, bgt.index))
bgt.head()

In [ ]:
blockpositions = pd.read_csv('blockmap-positions.csv', names=range(15))
blockpositions.head()

In [ ]:
flatrows = []
for y, colcities in blockpositions.iterrows():
    for x, city in colcities.iteritems():
        if isinstance(city, str):
            flatrows.append((x, y, city))

blockpositions_tbl = pd.DataFrame(flatrows, columns=('x', 'y', 'city')).set_index('city').sort_index()
bgtb = pd.merge(bgt, blockpositions_tbl, how='left', left_on='shortname', right_index=True)
bgtb.head()

In [ ]:
bgtb.to_csv('data_fastfood.csv')

In [ ]:
bgtb[bgtb['x'].apply(np.isnan)]

In [ ]:
plt.figure(figsize=(4, 3))
subcnt = bgt[['B', 'K', 'L']]
subcnt.columns = ['BurgerKing', 'KFC', 'Lotteria']
p = subcnt.sum(axis=0).plot(kind='bar')
plt.setp(p.get_xticklabels(), rotation=0)
plt.ylabel('number of )
plt.savefig('plot-shops-count.pdf')